In [ ]:
pip install llama-cpp-python


In [2]:
import tensorflow as tf
from tensorflow import keras
import os
import string
import chromadb
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from llama_cpp import Llama


In [3]:
documents = []
metadata = []
ids = []

with open('data/spells.txt', 'r', encoding='utf-8') as spells:
    for line in spells:
        line=line.split(" - ") #spells are separated by their description by a " - "
        metadata.append({'spell': line[0]}) #use the spell name as metadata. ChromaDB requires a dictionary here
        documents.append(line[1]) #use the description as documents
        ids.append(line[0]) #we also need an ID so use the spell name again.
print(ids)

['Aberto', 'Accio', 'Aguamenti', 'Alohomora', 'Anapneo', 'Aparecium', 'Apparate', 'Ascendio', 'Avada Kedavra', 'Avis', 'Bat-Bogey Hex', 'Bombardo', 'Brackium Emendo', 'Capacious Extremis', 'Confundo', 'Conjunctivitis Curse', 'Crinus Muto', 'Crucio', 'Diffindo', 'Disillusionment Charm', 'Disapparate', 'Engorgio', 'Episkey', 'Expecto patronum', 'Erecto', 'Evanesco', 'Expelliarmus', 'Ferula', 'Fidelius Charm', 'Fiendfyre Curse', 'Finite Incantatem', 'Furnunculus Curse', 'Geminio', 'Glisseo', 'Homenum Revelio', 'Homonculus Charm', 'Immobulus', 'Impedimenta', 'Incarcerous', '*Imperio', 'Impervius', 'Incendio', 'Langlock', 'Legilimens', 'Levicorpus', 'Locomotor Mortis', 'Lumos', 'Morsmordre', 'Mucus Ad Nauseam', 'Muffliato', 'Nox', 'Obliviate', 'Obscuro', 'Oculus Reparo', 'Oppugno', 'Petrificus Totalus', 'Periculum', 'Piertotum Locomotor', 'Protean Charm', 'Protego', 'Reducto', 'Reducio', 'Renneverate', 'Reparifors', 'Reparo', 'Rictusempra', 'Riddikulus', 'Scourgify', '*Sectumsempra', 'Serpe

In [4]:

client = chromadb.Client()

# create or open collection named 'spells' 
spells_collection = client.get_or_create_collection("spells")

# add spells to the collection, ignored for ids that already exist 
spells_collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

In [5]:
user_input="How do I create light"
results = spells_collection.query(
    query_texts=[user_input],
    n_results=1
)
results

{'ids': [['Lumos']],
 'distances': [[1.327547311782837]],
 'metadatas': [[{'spell': 'Lumos'}]],
 'embeddings': None,
 'documents': [["Illuminates the caster's wand\n"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [6]:
result=results['metadatas'][0][0]['spell']+" - " + results['documents'][0][0][:-1]
result

"Lumos - Illuminates the caster's wand"

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement llama-cpp (from versions: none)
ERROR: No matching distribution found for llama-cpp


In [10]:


llm = Llama.from_pretrained(
	repo_id="BramVanroy/GEITje-7B-ultra-GGUF",
	filename="geitje-7b-ultra-q8_0.gguf",
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)

ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
prompt = """
Recommended spell: {result}
User input: {user_input} 
Suggest a spell based on the recommended spell and the user input."""

prompt=prompt.format(user_input=user_input, result=result)
print ("prompt: ", prompt)
with tf.device('CPU: 0'): #Force on CPU for both apple silicon and low GPU mem configurations
        inference = model.generate(prompt, max_length = 100)
print(inference)